In [ ]:
# This is an tutorial example to apply EQCCT onto Japanese Earthquake Dataset.
# Prepared by Yangkang Chen and Omar M. Saad
# April, 2023

from eqcct.utils import *
import os
os.system('rm -rf ./detectionsJapan')

#     Please first download the mseed data from
#     https://drive.google.com/drive/folders/1jAkW4kOvwUDYxXW-ty3BTY81fjLF7sKE?usp=share_link
#
#     Then download the manual picks from
#     https://drive.google.com/drive/folders/1KK16j1-WbwqKfTvh5gJbzJrX0fiayhl6?usp=share_link
#
#     Then run the notebook 
#     notebooks/Test_Predict_MSEED_Japan.ipynb for the single-model (P and S together) example
#     notebooks/Test_Predict_MSEED_Japan_PS.ipynb for the double-model (P and S separately) example
#
#     make sure Japan_MSEED contains the TOWH folder
# 
# os.system('mkdir ./Japan_MSEED/')

json_basepath = os.path.join(os.getcwd(),"Example_Japan_station_list.json")


mseed_predictor(input_dir='./Japan_MSEED/',   
         input_model='../Model/test_trainer_016.h5',
         stations_json=json_basepath,
         output_dir='detectionsJapan',
         P_threshold=0.1,
         S_threshold=0.1, 
         number_of_plots=10,
         normalization_mode='std',
         batch_size=1,
         overlap=0.8,
         gpuid=None,
         gpu_limit=None) 

07-29 22:07 [INFO] [EQCCT_Picker] Running EQCCT_Picker
07-29 22:07 [INFO] [EQCCT_Picker] *** Loading the model ...
2024-07-29 22:07:42.558065: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 6000, 3)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 6000, 3)      102         ['input[0][0]']                  
                                                                                                  
 conv1d_33 (Conv1D)             (None, 6000, 3)      102         ['input[0][0]']                  
                                                                                                  
 batch_normalization (BatchNorm  (None, 6000, 3)     12          ['conv1d[0][0]']                 
 alization)                                                                                   

 activation_37 (Activation)     (None, 6000, 10)     0           ['batch_normalization_37[0][0]'] 
                                                                                                  
 add_1 (Add)                    (None, 6000, 10)     0           ['activation_4[0][0]',           
                                                                  'dropout[0][0]']                
                                                                                                  
 add_20 (Add)                   (None, 6000, 10)     0           ['activation_37[0][0]',          
                                                                  'dropout_19[0][0]']             
                                                                                                  
 conv1d_5 (Conv1D)              (None, 6000, 20)     2220        ['add_1[0][0]']                  
                                                                                                  
 conv1d_38

                                                                                                  
 patches_1 (Patches)            (None, None, 1600)   0           ['reshape_2[0][0]']              
                                                                                                  
 patch_encoder (PatchEncoder)   (None, 150, 40)      70040       ['patches[0][0]']                
                                                                                                  
 patch_encoder_1 (PatchEncoder)  (None, 150, 40)     70040       ['patches_1[0][0]']              
                                                                                                  
 conv1d_9 (Conv1D)              (None, 150, 40)      17640       ['patch_encoder[0][0]']          
                                                                                                  
 conv1d_42 (Conv1D)             (None, 150, 40)      17640       ['patch_encoder_1[0][0]']        
          

 activation_45 (Activation)     (None, 150, 40)      0           ['batch_normalization_45[0][0]'] 
                                                                                                  
 conv1d_13 (Conv1D)             (None, 150, 40)      17640       ['activation_12[0][0]']          
                                                                                                  
 conv1d_46 (Conv1D)             (None, 150, 40)      17640       ['activation_45[0][0]']          
                                                                                                  
 batch_normalization_13 (BatchN  (None, 150, 40)     160         ['conv1d_13[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_46 (BatchN  (None, 150, 40)     160         ['conv1d_46[0][0]']              
 ormalizat

...

                                                                                                  
 batch_normalization_58 (BatchN  (None, 150, 40)     160         ['conv1d_58[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_25 (Activation)     (None, 150, 40)      0           ['batch_normalization_25[0][0]'] 
                                                                                                  
 activation_58 (Activation)     (None, 150, 40)      0           ['batch_normalization_58[0][0]'] 
                                                                                                  
 add_12 (Add)                   (None, 150, 40)      0           ['activation_25[0][0]',          
                                                                  'multi_head_attention_2[0][0]'] 
          

 ormalization)                                                                                    
                                                                                                  
 batch_normalization_60 (BatchN  (None, 150, 40)     160         ['conv1d_60[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_27 (Activation)     (None, 150, 40)      0           ['batch_normalization_27[0][0]'] 
                                                                                                  
 activation_60 (Activation)     (None, 150, 40)      0           ['batch_normalization_60[0][0]'] 
                                                                                                  
 conv1d_28 (Conv1D)             (None, 150, 40)      17640       ['activation_27[0][0]']          
          

 activation_31 (Activation)     (None, 150, 40)      0           ['batch_normalization_31[0][0]'] 
                                                                                                  
 activation_64 (Activation)     (None, 150, 40)      0           ['batch_normalization_64[0][0]'] 
                                                                                                  
 add_16 (Add)                   (None, 150, 40)      0           ['activation_31[0][0]',          
                                                                  'multi_head_attention_3[0][0]'] 
                                                                                                  
 add_35 (Add)                   (None, 150, 40)      0           ['activation_64[0][0]',          
                                                                  'multi_head_attention_7[0][0]'] 
                                                                                                  
 conv1d_32

                                                                                                  
 picker_P (Conv1D)              (None, 6000, 1)      16          ['reshape_1[0][0]']              
                                                                                                  
 picker_S (Conv1D)              (None, 6000, 1)      16          ['reshape_3[0][0]']              
                                                                                                  
Total params: 1,277,568
Trainable params: 1,273,184
Non-trainable params: 4,384
__________________________________________________________________________________________________


07-29 22:07 [DEBUG] [h5py._conv] Creating converter from 3 to 5
07-29 22:07 [INFO] [EQCCT_Picker] *** Loading is complete!
07-29 22:07 [INFO] [EQCCT_Picker] There are files for 1 stations in ./Japan_MSEED/ directory.
07-29 22:07 [INFO] [EQCCT_Picker] Started working on TOWH, 1 out of 1 ...
07-29 22:07 [INFO] [EQCCT_Picker] 2014-01-01T00_43_59.210000Z__2014-01-01T00_45_59.210000Z.mseed
07-29 22:07 [INFO] [EQCCT_Picker] 2014-01-01T00_46_26.740000Z__2014-01-01T00_48_26.740000Z.mseed
07-29 22:07 [INFO] [EQCCT_Picker] 2014-01-01T02_43_15.520000Z__2014-01-01T02_45_15.520000Z.mseed
07-29 22:07 [INFO] [EQCCT_Picker] 2014-01-01T03_35_12.240000Z__2014-01-01T03_37_12.240000Z.mseed
07-29 22:07 [INFO] [EQCCT_Picker] 2014-01-01T04_40_26.690000Z__2014-01-01T04_42_26.690000Z.mseed
07-29 22:07 [INFO] [EQCCT_Picker] 2014-01-01T06_20_11.840000Z__2014-01-01T06_22_11.840000Z.mseed
07-29 22:07 [INFO] [EQCCT_Picker] 2014-01-01T06_40_17.620000Z__2014-01-01T06_42_17.620000Z.mseed
07-29 22:07 [INFO] [EQCCT_Pick

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('./detectionsJapan/TOWH_outputs/X_prediction_results.csv', low_memory=False)
data

In [ ]:
import obspy
finalnameall=[]
Parrivalfinal=[]
Pprobfinal=[]

name = (np.unique(data['file_name']))

for u in range(0,len(name)):
    
    exist = np.where(data['file_name']==name[u])[0]
    Parrival = np.array(data['p_arrival_time'][exist]) 
    Pprob = np.array(data['p_probability'][exist])
    
    PprobA = []
    ParrivalA = []
    for ku,zu in zip(Parrival,Pprob):
        if len(str(ku))>4:
            ParrivalA.append(ku)
            PprobA.append(zu)
            
    if len(PprobA)>0:
        #flagind = np.argmax(PprobA)

        flagind = 0
        finalnameall.append(name[u])
        Parrivalfinal.append(ParrivalA[flagind])
        Pprobfinal.append(PprobA[flagind])
        
    else:
        finalnameall.append(name[u])
        Parrivalfinal.append('nan')
        Pprobfinal.append('nan')

In [ ]:
## Make sure Japan_Catalog_Picks contains all these files
# https://drive.google.com/drive/folders/1KK16j1-WbwqKfTvh5gJbzJrX0fiayhl6?usp=share_link

PWaveTime   = np.load('./Japan_Catalog_Picks/PWaveTime.npy',allow_pickle=True)
SWaveTime   = np.load('./Japan_Catalog_Picks/SWaveTime.npy',allow_pickle=True)
TOWMAGall   = np.load('./Japan_Catalog_Picks/TOWMAGall.npy')
TOWDEPTHall = np.load('./Japan_Catalog_Picks/TOWDEPTHall.npy')
TOWLATall   = np.load('./Japan_Catalog_Picks/TOWLATall.npy')
TOWLONGall  = np.load('./Japan_Catalog_Picks/TOWLONGall.npy')
filenameallREAD = np.load(r'./Japan_Catalog_Picks/filenameall.npy')

PCAT=[]
for i in PWaveTime:
    PCAT.append(obspy.UTCDateTime(i))
PCAT = np.array(PCAT)

SCAT=[]
for i in SWaveTime:
    SCAT.append(obspy.UTCDateTime(i))
SCAT = np.array(SCAT)

len(PCAT), len(SCAT)


In [ ]:
# print(filenameall)
# SCAT[0]
len(finalnameall)

In [ ]:
print(filenameallREAD) ##filenameallREAD is from catalog files

In [ ]:
# print(finalnameall)    ##finalnameall is from EQCCT picking

In [ ]:
##filenameallREAD is from catalog files
##finalnameall is from EQCCT picking
## filenameallREAD and finalnameall have different formats so we need to create a different 
# finalnameall or filenameallREAD
# It is easier to change "filenameallREAD" by
filenameallREAD=[ii.replace(":", "_" ) for ii in filenameallREAD]
# print(filenameallREAD)

In [ ]:
countp = []
for g in Parrivalfinal:
    if len(str(g))<10:
        countp.append(g)
len(countp)    

In [ ]:
samp = 50
PEQ_err = []
miss=[]
mag=[]
dep=[]
filenameallREAD = np.array(filenameallREAD)
finalnameall = np.array(finalnameall)
missname=[]
PEQ_errName=[]
PEQ_errVal=[]
for i in range(len(filenameallREAD)):
    #temp = data['file_name'][i]
    temp = filenameallREAD[i]
    
    try:
        indxPick = np.where(finalnameall==temp)[0]
        PEQ_err.append(PCAT[i]-obspy.UTCDateTime(Parrivalfinal[indxPick[0]]))
        mag.append(TOWMAGall[indxPick])
        dep.append(TOWDEPTHall[indxPick])
        
        zo = PCAT[i]-obspy.UTCDateTime(Parrivalfinal[indxPick[0]])
        if zo>(samp/100):
            PEQ_errName.append(filenameallREAD[i])
            PEQ_errVal.append(zo)
            
    except:
        miss.append(filenameallREAD[i])
        #missname.append(i)
        continue
        
mag = np.array(mag)
dep = np.array(dep)
PEQ_err = np.array(PEQ_err)*100
len(PEQ_err), len(np.where(np.abs(PEQ_err)<=samp)[0]), len(miss)

In [ ]:
len(np.where(np.abs(PEQ_err)<=samp)[0]) /len(PEQ_err)

In [ ]:
a0 = np.where(np.abs(PEQ_err)<=samp)[0]
np.mean(PEQ_err[a0]/100), np.mean(np.abs(PEQ_err[a0])/100), np.std(PEQ_err[a0]/100)

In [ ]:
#003 (0.015358418017028289, 0.03775885745674265, 0.050380472932146377,  92.64, 92.34)
#006 (0.024432040189785093, 0.04585542841194530, 0.056218194597406401,  91.11, 90.87)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(PEQ_err/100, bins=2000)
plt.xlim([-1,1])
1

In [ ]:
TP = len(np.where(np.abs(PEQ_err)<=samp)[0])
TN = 0
FP = 0
FN = len(miss) + len(np.where(np.abs(PEQ_err)>samp)[0])
R = TP /(TP + FN)
TP+TN+FP+FN, R

In [ ]:
len(PEQ_err) + len(miss)

In [ ]:
import obspy
finalnameall=[]
Sarrivalfinal=[]
Sprobfinal=[]

name = (np.unique(data['file_name']))

for u in range(0,len(name)):
    
    exist = np.where(data['file_name']==name[u])[0]
    Sarrival = np.array(data['s_arrival_time'][exist]) 
    Sprob = np.array(data['s_probability'][exist])
    
    SprobA = []
    SarrivalA = []
    for ku,zu in zip(Sarrival,Sprob):
        if len(str(ku))>4:
            SarrivalA.append(ku)
            SprobA.append(zu)
            
    if len(SprobA)>0:
        #flagind = np.argmax(SprobA)
        flagind = 0
        finalnameall.append(name[u])
        Sarrivalfinal.append(SarrivalA[flagind])
        Sprobfinal.append(SprobA[flagind])
        
    else:
        finalnameall.append(name[u])
        Sarrivalfinal.append('nan')
        Sprobfinal.append('nan')

In [ ]:
counts = []
for g in Sarrivalfinal:
    if len(str(g))<10:
        counts.append(g)
len(counts)    

In [ ]:
samp = 50
SEQ_err = []
miss=[]
mag=[]
dep=[]
filenameallREAD = np.array(filenameallREAD)
finalnameall = np.array(finalnameall)
#for i in range(len(finalnameall)):
#    finalnameall[i] = finalnameall[i].replace('_',':')
#    finalnameall[i] = finalnameall[i].replace('::','__')
    

missname=[]
for i in range(len(filenameallREAD)):
    #temp = data['file_name'][i]
    temp = filenameallREAD[i]
    
    try:
        indxPick = np.where(finalnameall==temp)[0]
        SEQ_err.append(SCAT[i]-obspy.UTCDateTime(Sarrivalfinal[indxPick[0]]))
        mag.append(TOWMAGall[indxPick])
        dep.append(TOWDEPTHall[indxPick])
    except:
        miss.append(filenameallREAD[i])
        #missname.append(i)
        continue
        
mag = np.array(mag)
dep = np.array(dep)
SEQ_err = np.array(SEQ_err)*100
len(SEQ_err), len(np.where(np.abs(SEQ_err)<=samp)[0]), len(miss)

In [ ]:
len(np.where(np.abs(SEQ_err)<=samp)[0]) /len(SEQ_err)

In [ ]:
a0 = np.where(np.abs(SEQ_err)<=samp)[0]
np.mean(SEQ_err[a0]/100), np.mean(np.abs(SEQ_err[a0])/100), np.std(SEQ_err[a0]/100)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(SEQ_err/100, bins=2000)
plt.xlim([-1,1])
1

In [ ]:
TP = len(np.where(np.abs(SEQ_err)<=samp)[0])
TN = 0
FP = 0
FN =  len(miss) + len(np.where(np.abs(SEQ_err)>samp)[0])
R = TP /(TP + FN)
TP+TN+FP+FN, R